**DAEN690_Team Pass

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import boto3
from sagemaker import get_execution_role
import sagemaker.amazon.common as smac

**Dataset Disposals 

In [2]:
role = get_execution_role()
bucket = 'dean690-dataset'
data_key = 'gmu_training_disposals.csv'
data_path = 's3://{}/{}'.format(bucket, data_key)

# read csv file
data_disposals = pd.read_csv(data_path, low_memory = False)
data_disposals.head()

# read dataset information
data_disposals.info()

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20210312T142701 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78374 entries, 0 to 78373
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    78374 non-null  int64  
 1   BUSINESS_UNIT                 78374 non-null  object 
 2   DESCR                         78374 non-null  object 
 3   ASSET_SUBTYPE                 78374 non-null  object 
 4   ON_DS132                      78374 non-null  object 
 5   EXPECTED_RETIREMENT_DATE_REC  78374 non-null  object 
 6   RETIREMENT_DATE               78374 non-null  object 
 7   SCAN_TYPE                     72811 non-null  object 
 8   COST                          78374 non-null  float64
 9   LAST_USER_TO_COMMENT          78374 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 6.0+ MB


In [3]:
# remove duplicate rows
data_disposals.drop_duplicates(inplace=True)

# check if there is missing values
data_disposals.isnull().sum()

Unnamed: 0                         0
BUSINESS_UNIT                      0
DESCR                              0
ASSET_SUBTYPE                      0
ON_DS132                           0
EXPECTED_RETIREMENT_DATE_REC       0
RETIREMENT_DATE                    0
SCAN_TYPE                       5563
COST                               0
LAST_USER_TO_COMMENT               0
dtype: int64

**Scan_Type

In [5]:
# check column"Scan_type" value
data_disposals["SCAN_TYPE"].value_counts()

Scanned          67806
Manual            4292
Scan Transfer      440
Discovered         270
Invalid              3
Name: SCAN_TYPE, dtype: int64

In [6]:
# fill the missing value with 1
data_disposals["SCAN_TYPE"].fillna(1, inplace=True)

# adjust the values
data_disposals["SCAN_TYPE"].replace(["Invalid","Discovered","Scan Transfer","Manual","Scanned"]
                                    , [1,0.5,0.5,0.5,0], inplace=True)
# display the result
data_disposals["SCAN_TYPE"].value_counts()

0.0    67806
1.0     5566
0.5     5002
Name: SCAN_TYPE, dtype: int64

**EXPECTED_RETIREMENT_DATE_REC RETIREMENT_DATE

In [7]:
# adjust the data formate in each column
data_disposals["EXPECTED_RETIREMENT_DATE_REC"] = pd.to_datetime(data_disposals["EXPECTED_RETIREMENT_DATE_REC"])
data_disposals["RETIREMENT_DATE"] = pd.to_datetime(data_disposals["RETIREMENT_DATE"])

# create new column to store the results
data_disposals["time"] = data_disposals["EXPECTED_RETIREMENT_DATE_REC"] - data_disposals["RETIREMENT_DATE"]

# create new column to store the adjusted value 
data_disposals["ret_pfm"] = [1 if d > 0 else 0  for d in data_disposals["time"].dt.days]

# display value of the adjusted column "ret_pfm"
data_disposals["ret_pfm"].value_counts()

0    43159
1    35215
Name: ret_pfm, dtype: int64

In [8]:
# extract the year in column 'EXPECTED_RETIREMENT_DATE_REC' and 'RETIREMENT_DATE'
data_disposals['exp_ret_year'] = data_disposals['EXPECTED_RETIREMENT_DATE_REC'].apply(lambda r:r.year)
data_disposals['act_ret_year'] = data_disposals['RETIREMENT_DATE'].apply(lambda r:r.year)

# readjust the date
data_disposals["ret_pfm"][data_disposals['exp_ret_year'] == data_disposals['act_ret_year']] = 0

# display the adjusted value
data_disposals['ret_pfm'].value_counts()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0    45391
1    32983
Name: ret_pfm, dtype: int64

***ON_DS132

In [15]:
data_disposals['ON_DS132'].value_counts()

Yes    75227
No      3147
Name: ON_DS132, dtype: int64

In [16]:
# adjust the value
data_disposals["ON_DS132"].replace(["Yes", "No"], [0,1], inplace=True)

# display the adjusted value
data_disposals['ON_DS132'].value_counts()

0    75227
1     3147
Name: ON_DS132, dtype: int64

**COST

In [17]:
# convert datatype to string
data_disposals["COST"] = data_disposals["COST"].astype(str)

# remove symbol in cost column
data_disposals.COST = data_disposals.COST.str.replace('[#,@,&,-]', '')
data_disposals.COST = data_disposals.COST.str.replace("'", '')

# convert datatype to float
data_disposals["COST"] = data_disposals["COST"].astype(float)

# display the result
data_disposals.head()

,Unnamed: 0,BUSINESS_UNIT,DESCR,ASSET_SUBTYPE,ON_DS132,EXPECTED_RETIREMENT_DATE_REC,RETIREMENT_DATE,SCAN_TYPE,COST,LAST_USER_TO_COMMENT,time,ret_pfm,exp_ret_year,act_ret_year
0,1,LOCATION 62,VACUUM CLEANER,Appliances,0,2014-01-10,2018-10-15,0.0,75.00,USER1,-1739 days,0,2014,2018
1,2,LOCATION 32,CPU (CENTRAL PROCESSOR UNIT),CPU,0,2016-08-15,2018-11-10,0.0,699.99,USER2,-817 days,0,2016,2018
2,3,LOCATION 82,CPU (CENTRAL PROCESSOR UNIT),CPU,0,2015-04-12,2019-03-21,0.0,676.20,USER3,-1439 days,0,2015,2019
3,4,LOCATION 82,MATTRESS TWIN,Household Furniture,0,2017-03-20,2019-05-08,0.0,200.00,USER3,-779 days,0,2017,2019
4,5,LOCATION 124,CHAIR HSLD EASY,Household Furniture,0,2021-10-08,2019-03-20,0.0,683.00,USER4,933 days,1,2021,2019


**Generate processed table and filter error cost

In [18]:
# select colomns
processed_disposals_df = data_disposals[['BUSINESS_UNIT', 'LAST_USER_TO_COMMENT','SCAN_TYPE', 'ret_pfm'
                                         , 'ON_DS132','COST']]

# rename the columns
processed_disposals_df.columns = ['location', 'user','scan_type_pfm', 'ret_date_pfm'
                                  , 'disp_doc','ori_cost']

# display the processed table
processed_disposals_df

,location,user,scan_type_pfm,ret_date_pfm,disp_doc,ori_cost
0,LOCATION 62,USER1,0.0,0,0,75.00
1,LOCATION 32,USER2,0.0,0,0,699.99
2,LOCATION 82,USER3,0.0,0,0,676.20
3,LOCATION 82,USER3,0.0,0,0,200.00
4,LOCATION 124,USER4,0.0,1,0,683.00
...,...,...,...,...,...,...
78369,LOCATION 131,USER44,0.0,0,0,522.00
78370,LOCATION 131,USER44,0.0,1,0,811.38
78371,LOCATION 131,USER44,0.0,0,0,229.42
78372,LOCATION 131,USER44,0.0,1,1,871.45


In [19]:
# copy the processed table
processed_disposals_df_1 = processed_disposals_df.copy()

# filter the error cost
processed_disposals_df_1.loc[(
    processed_disposals_df_1['scan_type_pfm']>0) | (
    processed_disposals_df_1['ret_date_pfm']>0) | (
    processed_disposals_df_1['disp_doc']>0), 'err_cost'] = processed_disposals_df_1['ori_cost']

# check if there is missing values
processed_disposals_df_1.isnull().sum()

location             0
user                 0
scan_type_pfm        0
ret_date_pfm         0
disp_doc             0
ori_cost             0
err_cost         38818
dtype: int64

In [20]:
# fill nan value with 0
processed_disposals_df_1.fillna(0, inplace=True)

# check if the missiving values are removed
processed_disposals_df_1.isnull().sum()

location         0
user             0
scan_type_pfm    0
ret_date_pfm     0
disp_doc         0
ori_cost         0
err_cost         0
dtype: int64

In [21]:
# display the first five rows
processed_disposals_df_1.head()

,location,user,scan_type_pfm,ret_date_pfm,disp_doc,ori_cost,err_cost
0,LOCATION 62,USER1,0.0,0,0,75.00,0.0
1,LOCATION 32,USER2,0.0,0,0,699.99,0.0
2,LOCATION 82,USER3,0.0,0,0,676.20,0.0
3,LOCATION 82,USER3,0.0,0,0,200.00,0.0
4,LOCATION 124,USER4,0.0,1,0,683.00,683.0


**Calculation: Sum the transactions number of each action and the cost

In [22]:
# create new table to continue the modeling
disposals_rating_df = pd.DataFrame(columns=["user", "scan_type_#", "ret_date_#", "disp_doc_#"
                                            , "org_cost_disposals","err_cost_disposals"
                                            ,"scan_type_%", "ret_date_%", "disp_doc_%", "err_cost_disposals_%"])

# use sum() function to calculate total amounts in each column
# scan_type_# : the total transaction number of action "SCAN_TYPE"
# ret_date_# : the total transaction number of action "retirement date"
# disp_doc_# : the total transaction number of action "disposal document"
# org_cost_disposals : total orginal cost by user
# err_cost_disposals : total error cost by user
# scan_type_% : the proportion of error transaction number of action "SCAN_TYPE"
# ret_date_% : the proportion of error transaction number of action "retirement date"
# disp_doc_% : the proportion of error transaction number of action "disposal document"
# err_cost_disposals_% : the proportion of error cost by user

for index, row in processed_disposals_df_1.groupby('user'):
    sub_df = pd.DataFrame({"user": index,  "scan_type_#": row["scan_type_pfm"].sum()
                           , "ret_date_#": row["ret_date_pfm"].sum()
                           , "disp_doc_#": row["disp_doc"].sum()
                           , "org_cost_disposals": row["ori_cost"].sum()
                           , "err_cost_disposals": row["err_cost"].sum()
                           , "scan_type_%": row["scan_type_pfm"].sum()/len(row)
                           , "ret_date_%": row["ret_date_pfm"].sum()/len(row)
                           , "disp_doc_%": row["disp_doc"].sum()/len(row)
                           , "err_cost_disposals_%": row["err_cost"].sum()/row["ori_cost"].sum()}, index=[0])
    disposals_rating_df = pd.concat([disposals_rating_df, sub_df], axis=0)

# display the result
disposals_rating_df

,user,scan_type_#,ret_date_#,disp_doc_#,org_cost_disposals,err_cost_disposals,scan_type_%,ret_date_%,disp_doc_%,err_cost_disposals_%
0,USER1,26.0,377,87,353152.910,162311.120,0.030481,0.441970,0.101993,0.459606
0,USER10,99.0,208,0,376585.870,142545.810,0.131300,0.275862,0.000000,0.378521
0,USER100,14.5,83,0,279244.708,202081.708,0.064159,0.367257,0.000000,0.723672
0,USER101,42.0,140,0,142507.140,76011.120,0.172840,0.576132,0.000000,0.533385
0,USER102,24.0,93,0,170262.852,46378.746,0.049383,0.191358,0.000000,0.272395
...,...,...,...,...,...,...,...,...,...,...
0,USER95,78.0,119,0,78416.320,78416.320,0.655462,1.000000,0.000000,1.000000
0,USER96,36.5,22,0,27658.430,17604.770,0.344340,0.207547,0.000000,0.636506
0,USER97,19.0,168,0,127782.660,59091.890,0.049351,0.436364,0.000000,0.462441
0,USER98,3.5,8,0,32666.590,2694.230,0.021212,0.048485,0.000000,0.082477


In [23]:
# create a new variable to store reference columns
cost_disp_temp = disposals_rating_df[['user','org_cost_disposals'
                                      ,'scan_type_%','ret_date_%','disp_doc_%','err_cost_disposals_%']]

# filter the columns before modeling
disposals_rating_df = disposals_rating_df [['user','scan_type_#','ret_date_#','disp_doc_#','err_cost_disposals']]

# display the table
disposals_rating_df

,user,scan_type_#,ret_date_#,disp_doc_#,err_cost_disposals
0,USER1,26.0,377,87,162311.120
0,USER10,99.0,208,0,142545.810
0,USER100,14.5,83,0,202081.708
0,USER101,42.0,140,0,76011.120
0,USER102,24.0,93,0,46378.746
...,...,...,...,...,...
0,USER95,78.0,119,0,78416.320
0,USER96,36.5,22,0,17604.770
0,USER97,19.0,168,0,59091.890
0,USER98,3.5,8,0,2694.230


**DATA MODELING

In [24]:
# weights for role disposals
weight_scan_type = 1/15
weight_ret_date = 0.5/15
weight_disp_doc = 1/15
weight_disp_cost = 5/15

In [25]:
# Standardize the error cost
disposals_rating_df["std_err_cost_disposals"] = StandardScaler().fit_transform(
    disposals_rating_df["err_cost_disposals"].astype(float).values.reshape(-1, 1))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
# Normalizing each feature
X = MinMaxScaler(feature_range=(0, 1)).fit_transform(disposals_rating_df[["scan_type_#", "ret_date_#"
                                                                          , "disp_doc_#","std_err_cost_disposals"]])
# assign weights for each attribute
X[:, 0] = weight_scan_type * X[:, 0]    # weight for scan_type_# 
X[:, 1] = weight_ret_date * X[:, 1]    # weight for ret_date_pfm 
X[:, 2] = weight_disp_doc * X[:, 2]    # weight for disp_doc_#
X[:, 3] = weight_disp_cost * X[:, 3]   # weight for std_err_cost_disposals



rating =  X[:,0] + X[:,1] + X[:,2] + X[:,3]
disposals_rating_df["disposals_rating"] = rating

# display the result
disposals_rating_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,user,scan_type_#,ret_date_#,disp_doc_#,err_cost_disposals,std_err_cost_disposals,disposals_rating
0,USER1,26.0,377,87,162311.120,0.449160,0.060155
0,USER10,99.0,208,0,142545.810,0.323838,0.051490
0,USER100,14.5,83,0,202081.708,0.701325,0.061813
0,USER101,42.0,140,0,76011.120,-0.098024,0.027486
0,USER102,24.0,93,0,46378.746,-0.285908,0.016842
...,...,...,...,...,...,...,...
0,USER95,78.0,119,0,78416.320,-0.082774,0.029684
0,USER96,36.5,22,0,17604.770,-0.468349,0.007584
0,USER97,19.0,168,0,59091.890,-0.205300,0.021873
0,USER98,3.5,8,0,2694.230,-0.562889,0.001146


**RESULT

In [30]:
# merge the orginal total cost to the result for reference
disposals_rating_df = pd.merge(disposals_rating_df, cost_disp_temp, on="user", how="outer")

# show the results by score from large to small
disposals_rating_df.sort_values(by='disposals_rating', ascending=False)

,user,scan_type_#,ret_date_#,disp_doc_#,err_cost_disposals,std_err_cost_disposals,disposals_rating,org_cost_disposals,scan_type_%,ret_date_%,disp_doc_%,err_cost_disposals_%
157,USER44,105.0,1507,249,1135946.311,6.622482,0.379776,1764655.665,0.033302,0.477958,0.078972,0.643721
178,USER63,1242.5,1594,6,912374.690,5.204929,0.367952,1154820.350,0.445181,0.571121,0.002150,0.790058
179,USER64,167.5,1321,0,1016052.240,5.862295,0.334763,1386107.310,0.096486,0.760945,0.000000,0.733026
167,USER53,195.5,1538,1786,628189.278,3.403055,0.293655,660450.721,0.096022,0.755403,0.877210,0.951152
149,USER37,432.0,1019,107,551575.870,2.917289,0.210337,650603.510,0.313043,0.738406,0.077536,0.847791
...,...,...,...,...,...,...,...,...,...,...,...,...
114,USER201,0.0,0,0,0.000,-0.579972,0.000000,1482.340,0.000000,0.000000,0.000000,0.000000
97,USER187,0.0,0,0,0.000,-0.579972,0.000000,983.260,0.000000,0.000000,0.000000,0.000000
69,USER161,0.0,0,0,0.000,-0.579972,0.000000,537.660,0.000000,0.000000,0.000000,0.000000
91,USER181,0.0,0,0,0.000,-0.579972,0.000000,30751.470,0.000000,0.000000,0.000000,0.000000


In [31]:
# show the top 10 user who have the higher scores
disposals_top_10 = disposals_rating_df.nlargest(10,'disposals_rating')
disposals_top_10

,user,scan_type_#,ret_date_#,disp_doc_#,err_cost_disposals,std_err_cost_disposals,disposals_rating,org_cost_disposals,scan_type_%,ret_date_%,disp_doc_%,err_cost_disposals_%
157,USER44,105.0,1507,249,1135946.311,6.622482,0.379776,1764655.665,0.033302,0.477958,0.078972,0.643721
178,USER63,1242.5,1594,6,912374.690,5.204929,0.367952,1154820.350,0.445181,0.571121,0.002150,0.790058
179,USER64,167.5,1321,0,1016052.240,5.862295,0.334763,1386107.310,0.096486,0.760945,0.000000,0.733026
167,USER53,195.5,1538,1786,628189.278,3.403055,0.293655,660450.721,0.096022,0.755403,0.877210,0.951152
149,USER37,432.0,1019,107,551575.870,2.917289,0.210337,650603.510,0.313043,0.738406,0.077536,0.847791
152,USER4,258.5,928,0,584541.480,3.126307,0.204804,954106.460,0.136196,0.488936,0.000000,0.612659
163,USER5,123.5,845,0,469734.100,2.398372,0.162136,799476.260,0.076141,0.520962,0.000000,0.587552
186,USER70,150.5,922,0,452719.056,2.290488,0.160202,1557018.824,0.043826,0.268492,0.000000,0.290760
169,USER55,203.0,929,0,412404.729,2.034876,0.151336,739002.540,0.101551,0.464732,0.000000,0.558056
161,USER48,214.0,642,0,429852.945,2.145506,0.151044,602691.845,0.154290,0.462870,0.000000,0.713222


**DASHBOARD

In [32]:
# outer join two tables by using common field "user"
dashboard_disposals_df = processed_disposals_df[["location", "user"]]
dashboard_disposals_df = pd.merge(dashboard_disposals_df, disposals_rating_df
                                  , left_on="user", right_on="user" ,how="left")
# display the result
dashboard_disposals_df

,location,user,scan_type_#,ret_date_#,disp_doc_#,err_cost_disposals,std_err_cost_disposals,disposals_rating,org_cost_disposals,scan_type_%,ret_date_%,disp_doc_%,err_cost_disposals_%
0,LOCATION 62,USER1,26.0,377,87,162311.120,0.449160,0.060155,353152.910,0.030481,0.441970,0.101993,0.459606
1,LOCATION 32,USER2,13.0,256,273,218238.540,0.803767,0.080281,475456.501,0.013388,0.263646,0.281153,0.459008
2,LOCATION 82,USER3,21.0,207,0,107465.230,0.101410,0.036990,381297.880,0.028767,0.283562,0.000000,0.281841
3,LOCATION 82,USER3,21.0,207,0,107465.230,0.101410,0.036990,381297.880,0.028767,0.283562,0.000000,0.281841
4,LOCATION 124,USER4,258.5,928,0,584541.480,3.126307,0.204804,954106.460,0.136196,0.488936,0.000000,0.612659
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78369,LOCATION 131,USER44,105.0,1507,249,1135946.311,6.622482,0.379776,1764655.665,0.033302,0.477958,0.078972,0.643721
78370,LOCATION 131,USER44,105.0,1507,249,1135946.311,6.622482,0.379776,1764655.665,0.033302,0.477958,0.078972,0.643721
78371,LOCATION 131,USER44,105.0,1507,249,1135946.311,6.622482,0.379776,1764655.665,0.033302,0.477958,0.078972,0.643721
78372,LOCATION 131,USER44,105.0,1507,249,1135946.311,6.622482,0.379776,1764655.665,0.033302,0.477958,0.078972,0.643721


In [33]:
dashboard_disposals_df.isnull().sum()

location                  0
user                      0
scan_type_#               0
ret_date_#                0
disp_doc_#                0
err_cost_disposals        0
std_err_cost_disposals    0
disposals_rating          0
org_cost_disposals        0
scan_type_%               0
ret_date_%                0
disp_doc_%                0
err_cost_disposals_%      0
dtype: int64

In [34]:
# remove duplicates row
dashboard_disposals_df.drop_duplicates(['location','user','disposals_rating'],inplace=True)
dashboard_disposals_df

,location,user,scan_type_#,ret_date_#,disp_doc_#,err_cost_disposals,std_err_cost_disposals,disposals_rating,org_cost_disposals,scan_type_%,ret_date_%,disp_doc_%,err_cost_disposals_%
0,LOCATION 62,USER1,26.0,377,87,162311.12,0.449160,0.060155,353152.910,0.030481,0.441970,0.101993,0.459606
1,LOCATION 32,USER2,13.0,256,273,218238.54,0.803767,0.080281,475456.501,0.013388,0.263646,0.281153,0.459008
2,LOCATION 82,USER3,21.0,207,0,107465.23,0.101410,0.036990,381297.880,0.028767,0.283562,0.000000,0.281841
4,LOCATION 124,USER4,258.5,928,0,584541.48,3.126307,0.204804,954106.460,0.136196,0.488936,0.000000,0.612659
8,LOCATION 29,USER5,123.5,845,0,469734.10,2.398372,0.162136,799476.260,0.076141,0.520962,0.000000,0.587552
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57716,LOCATION 218,USER215,0.0,1,0,202.00,-0.578691,0.000080,202.000,0.000000,1.000000,0.000000,1.000000
60566,LOCATION 141,USER216,0.0,1,0,91.92,-0.579389,0.000048,91.920,0.000000,1.000000,0.000000,1.000000
62303,LOCATION 47,USER217,0.0,1,0,399.64,-0.577438,0.000138,399.640,0.000000,1.000000,0.000000,1.000000
64607,LOCATION 100,USER218,0.5,1,0,448.71,-0.577127,0.000179,448.710,0.500000,1.000000,0.000000,1.000000


In [35]:
# generate new dashboard
dashboard_disposals_df_1 = dashboard_disposals_df.copy()

# select displayed columns
dashboard_disposals_df_1 = dashboard_disposals_df_1[['location', 'user','disposals_rating'
                                                     , 'scan_type_%', 'ret_date_%', 'disp_doc_%', 'err_cost_disposals_%']]
# adjust the column names
dashboard_disposals_df_1['role'] = 'Disposals'
dashboard_disposals_df_1.columns.values[2] = "rating"

# display the results
dashboard_disposals_df_1

,location,user,rating,scan_type_%,ret_date_%,disp_doc_%,err_cost_disposals_%,role
0,LOCATION 62,USER1,0.060155,0.030481,0.441970,0.101993,0.459606,Disposals
1,LOCATION 32,USER2,0.080281,0.013388,0.263646,0.281153,0.459008,Disposals
2,LOCATION 82,USER3,0.036990,0.028767,0.283562,0.000000,0.281841,Disposals
4,LOCATION 124,USER4,0.204804,0.136196,0.488936,0.000000,0.612659,Disposals
8,LOCATION 29,USER5,0.162136,0.076141,0.520962,0.000000,0.587552,Disposals
...,...,...,...,...,...,...,...,...
57716,LOCATION 218,USER215,0.000080,0.000000,1.000000,0.000000,1.000000,Disposals
60566,LOCATION 141,USER216,0.000048,0.000000,1.000000,0.000000,1.000000,Disposals
62303,LOCATION 47,USER217,0.000138,0.000000,1.000000,0.000000,1.000000,Disposals
64607,LOCATION 100,USER218,0.000179,0.500000,1.000000,0.000000,1.000000,Disposals


In [36]:
# select the records that rating >= 0
dashboard_disposals_df_2 = dashboard_disposals_df_1[dashboard_disposals_df_1['rating'] >= 0]
dashboard_disposals_df_2

,location,user,rating,scan_type_%,ret_date_%,disp_doc_%,err_cost_disposals_%,role
0,LOCATION 62,USER1,0.060155,0.030481,0.441970,0.101993,0.459606,Disposals
1,LOCATION 32,USER2,0.080281,0.013388,0.263646,0.281153,0.459008,Disposals
2,LOCATION 82,USER3,0.036990,0.028767,0.283562,0.000000,0.281841,Disposals
4,LOCATION 124,USER4,0.204804,0.136196,0.488936,0.000000,0.612659,Disposals
8,LOCATION 29,USER5,0.162136,0.076141,0.520962,0.000000,0.587552,Disposals
...,...,...,...,...,...,...,...,...
57716,LOCATION 218,USER215,0.000080,0.000000,1.000000,0.000000,1.000000,Disposals
60566,LOCATION 141,USER216,0.000048,0.000000,1.000000,0.000000,1.000000,Disposals
62303,LOCATION 47,USER217,0.000138,0.000000,1.000000,0.000000,1.000000,Disposals
64607,LOCATION 100,USER218,0.000179,0.500000,1.000000,0.000000,1.000000,Disposals


**DATASET LOCATIONS 

In [37]:
# read excel file
role = get_execution_role()
bucket = 'dean690-dataset'
data_key2 = 'gmu_training_locations.csv'
data_path2 = 's3://{}/{}'.format(bucket, data_key2)

# read csv file
data_locations = pd.read_csv(data_path2, low_memory = False)
data_locations.head()


# read data information
data_locations.info()

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20210312T142701 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115341 entries, 0 to 115340
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Unnamed: 0        115341 non-null  int64 
 1   BUSINESS_UNIT     115341 non-null  object
 2   DOS_AGENCY_CD     115341 non-null  object
 3   ASSET_SUBTYPE     115341 non-null  object
 4   LOCATION_DATE     115341 non-null  object
 5   DS584_NUM         115331 non-null  object
 6   LOCATION_TYPE     115341 non-null  object
 7   FORM_CHECK        94134 non-null   object
 8   ENTERED_BY        109796 non-null  object
 9   COST              115341 non-null  object
 10  LOCATION          115341 non-null  object
 11  VALID_DS584_FLAG  115341 non-null  int64 
dtypes: int64(2), object(10)
memory usage: 10.6+ MB


In [38]:
# check missing value
# in this case, the important attributes dont have any missing values, so we dont process anything on the missing data
data_locations.isnull().sum()

Unnamed: 0              0
BUSINESS_UNIT           0
DOS_AGENCY_CD           0
ASSET_SUBTYPE           0
LOCATION_DATE           0
DS584_NUM              10
LOCATION_TYPE           0
FORM_CHECK          21207
ENTERED_BY           5545
COST                    0
LOCATION                0
VALID_DS584_FLAG        0
dtype: int64

In [39]:
# remove duplicate rows
data_locations.drop_duplicates(inplace=True)

**VALID_DS584_FLAG

In [40]:
# check the values after adjustments
data_locations["VALID_DS584_FLAG"].value_counts()

1    100676
0     14665
Name: VALID_DS584_FLAG, dtype: int64

In [41]:
# adjust the values in column "VALID_DS584_FLAG"
data_locations["VALID_DS584_FLAG"] = [1 if i == 0 else 0 for i in data_locations["VALID_DS584_FLAG"].values]

# check the values after adjustments
data_locations["VALID_DS584_FLAG"].value_counts()

0    100676
1     14665
Name: VALID_DS584_FLAG, dtype: int64

**COST

In [42]:
# convert datatype to string
data_locations["COST"] = data_locations["COST"].astype(str)

# remove symbols in cost column
data_locations.COST = data_locations.COST.str.replace('[#,@,&,-]', '')
data_locations.COST = data_locations.COST.str.replace("'", '')

# convert datatype to float
data_locations["COST"] = data_locations["COST"].astype(float)

# display the result
data_locations.head()

,Unnamed: 0,BUSINESS_UNIT,DOS_AGENCY_CD,ASSET_SUBTYPE,LOCATION_DATE,DS584_NUM,LOCATION_TYPE,FORM_CHECK,ENTERED_BY,COST,LOCATION,VALID_DS584_FLAG
0,1,LOCATION 8,AGENCY1,Household Furniture,Thu Jun 13 2019 00:00:00 GMT-0400 (EDT),NXP19007396,RE,NXP19007396,USER38,327.05,L00001744,0
1,2,LOCATION 113,AGENCY1,Floor and Wall Coverings,Wed Sep 18 2019 00:00:00 GMT-0400 (EDT),NXP19001319,RE,NXP19001319,USER65,467.29,L00000559,0
2,3,LOCATION 66,AGENCY2,Appliances,Mon Jan 28 2019 00:00:00 GMT-0500 (EST),,RE,NaN,USER219,1084.03,L00010707,1
3,4,LOCATION 57,AGENCY1,Household Furniture,Thu Dec 13 2018 00:00:00 GMT-0500 (EST),NXP19005237,RE,NXP19005237,USER35,204.49,L00002673,0
4,5,LOCATION 15,AGENCY1,Appliances,Wed Jul 03 2019 00:00:00 GMT-0400 (EDT),NXP19004963,RE,NXP19004963,USER220,1006.50,L00000541,0


**Generate processed table and filter error cost

In [43]:
# select colomns
processed_locations_df = data_locations[['BUSINESS_UNIT', 'ENTERED_BY','VALID_DS584_FLAG', 'COST']]

# rename the columns
processed_locations_df.columns = ['location', 'user','val_ds584_flag', 'ori_cost']

# display the table
processed_locations_df

,location,user,val_ds584_flag,ori_cost
0,LOCATION 8,USER38,0,327.05
1,LOCATION 113,USER65,0,467.29
2,LOCATION 66,USER219,1,1084.03
3,LOCATION 57,USER35,0,204.49
4,LOCATION 15,USER220,0,1006.50
...,...,...,...,...
115336,LOCATION 101,USER248,1,727.96
115337,LOCATION 67,USER58,0,13864.05
115338,LOCATION 174,USER153,0,296.39
115339,LOCATION 218,USER224,0,1097.92


In [44]:
# copy the processed table
processed_locations_df_1 = processed_locations_df.copy()

# filter the error cost
processed_locations_df_1.loc[(processed_locations_df_1['val_ds584_flag']>0), 'err_cost'] = processed_locations_df_1['ori_cost']

# check if there is missing values
processed_locations_df_1.isnull().sum()

location               0
user                5545
val_ds584_flag         0
ori_cost               0
err_cost          100676
dtype: int64

In [45]:
# fill nan value with 0
processed_locations_df_1.fillna(0, inplace=True)

# check if there missing values are removed
processed_locations_df_1.isnull().sum()

location          0
user              0
val_ds584_flag    0
ori_cost          0
err_cost          0
dtype: int64

In [46]:
# display the results
processed_locations_df_1.head()

,location,user,val_ds584_flag,ori_cost,err_cost
0,LOCATION 8,USER38,0,327.05,0.00
1,LOCATION 113,USER65,0,467.29,0.00
2,LOCATION 66,USER219,1,1084.03,1084.03
3,LOCATION 57,USER35,0,204.49,0.00
4,LOCATION 15,USER220,0,1006.50,0.00


**Calculation: Sum the transactions number of each action and the cost

In [47]:
# create new table to continue the modeling
locations_rating_df = pd.DataFrame(columns=["user", "val_ds584_flag_#", "org_cost_locations"
                                            , "err_cost_locations"
                                            ,"val_ds584_flag_%", "err_cost_locations_%"])

# use sum() function to calculate total amounts in each column
# val_ds584_flag_# : the total transaction number of action "VAL_DS584_FLAG"
# org_cost_locations : total orginal cost by user
# err_cost_locations : total error cost by user
# val_ds584_flag_% : the proportion of error transaction number of action "VAL_DS584_FLAG"
# err_cost_locations_% : the proportion of error cost by user

for index, row in processed_locations_df_1.groupby('user'):
    sub_df = pd.DataFrame({"user": index,  "val_ds584_flag_#": row["val_ds584_flag"].sum()
                           ,"org_cost_locations": row["ori_cost"].sum()
                           , "err_cost_locations": row["err_cost"].sum()
                           , "val_ds584_flag_%": row["val_ds584_flag"].sum() / len(row)
                           , "err_cost_locations_%": row["err_cost"].sum() / row["ori_cost"].sum()}, index=[0])
    locations_rating_df = pd.concat([locations_rating_df, sub_df], axis=0)
    
# display the result
locations_rating_df

,user,val_ds584_flag_#,org_cost_locations,err_cost_locations,val_ds584_flag_%,err_cost_locations_%
0,0,0,2879363.745,0.000,0.000000,0.000000
0,USER1,34,121126.220,9316.030,0.143460,0.076912
0,USER10,123,201355.902,54728.190,0.261702,0.271798
0,USER101,11,54829.500,3596.160,0.099099,0.065588
0,USER102,98,286517.269,36213.212,0.137447,0.126391
...,...,...,...,...,...,...
0,USER9,18,21774.240,7945.970,0.321429,0.364925
0,USER90,3,4306.640,2449.890,0.333333,0.568863
0,USER93,0,1840.000,0.000,0.000000,0.000000
0,USER96,111,83037.940,52714.990,0.637931,0.634830


In [48]:
# create a new variable to store reference columns
cost_loc_temp = locations_rating_df[['user','org_cost_locations'
                                     ,'val_ds584_flag_%','err_cost_locations_%']]

# filter the columns before modeling
locations_rating_df = locations_rating_df [['user','val_ds584_flag_#','err_cost_locations']]

# display the table
locations_rating_df

,user,val_ds584_flag_#,err_cost_locations
0,0,0,0.000
0,USER1,34,9316.030
0,USER10,123,54728.190
0,USER101,11,3596.160
0,USER102,98,36213.212
...,...,...,...
0,USER9,18,7945.970
0,USER90,3,2449.890
0,USER93,0,0.000
0,USER96,111,52714.990


**DATA MODELING

In [49]:
# weights for role locations
weight_val_flag = 1/15
weight_loc_cost = 2/15

In [50]:
# Standardize the error cost
locations_rating_df["std_err_cost_locations"] = StandardScaler().fit_transform(
    locations_rating_df["err_cost_locations"].astype(float).values.reshape(-1, 1))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [51]:
# Normalizing each feature
X_1 = MinMaxScaler(feature_range=(0, 1)).fit_transform(locations_rating_df[["val_ds584_flag_#", "std_err_cost_locations"]]) 

# assign weights for each attribute
X_1[:, 0] = weight_val_flag * X_1[:, 0]    # weight for val_ds584_flag_# 
X_1[:, 1] = weight_loc_cost * X_1[:, 1]    # weight for std_err_cost_locations


rating_1 = X_1[:,0] + X_1[:,1]
locations_rating_df["locations_rating"] = rating_1

# display the result
locations_rating_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,user,val_ds584_flag_#,err_cost_locations,std_err_cost_locations,locations_rating
0,0,0,0.000,-0.485544,0.000000
0,USER1,34,9316.030,-0.301520,0.006051
0,USER10,123,54728.190,0.595527,0.027642
0,USER101,11,3596.160,-0.414507,0.002117
0,USER102,98,36213.212,0.229792,0.020002
...,...,...,...,...,...
0,USER9,18,7945.970,-0.328583,0.004028
0,USER90,3,2449.890,-0.437150,0.000979
0,USER93,0,0.000,-0.485544,0.000000
0,USER96,111,52714.990,0.555759,0.025855


**RESULTS

In [53]:
# merge the orginal total cost to the result for reference
locations_rating_df = pd.merge(locations_rating_df, cost_loc_temp, on="user", how="outer")
locations_rating_df

,user,val_ds584_flag_#,err_cost_locations,std_err_cost_locations,locations_rating,org_cost_locations,val_ds584_flag_%,err_cost_locations_%
0,0,0,0.000,-0.485544,0.000000,2879363.745,0.000000,0.000000
1,USER1,34,9316.030,-0.301520,0.006051,121126.220,0.143460,0.076912
2,USER10,123,54728.190,0.595527,0.027642,201355.902,0.261702,0.271798
3,USER101,11,3596.160,-0.414507,0.002117,54829.500,0.099099,0.065588
4,USER102,98,36213.212,0.229792,0.020002,286517.269,0.137447,0.126391
...,...,...,...,...,...,...,...,...
326,USER9,18,7945.970,-0.328583,0.004028,21774.240,0.321429,0.364925
327,USER90,3,2449.890,-0.437150,0.000979,4306.640,0.333333,0.568863
328,USER93,0,0.000,-0.485544,0.000000,1840.000,0.000000,0.000000
329,USER96,111,52714.990,0.555759,0.025855,83037.940,0.637931,0.634830


In [54]:
# remove the the first row
locations_rating_df.drop([0], inplace = True)

# show the results by score from large to small
locations_rating_df.sort_values(by='locations_rating', ascending=False)

,user,val_ds584_flag_#,err_cost_locations,std_err_cost_locations,locations_rating,org_cost_locations,val_ds584_flag_%,err_cost_locations_%
108,USER270,190,487493.270,9.144131,0.152911,487493.270,1.000000,1.000000
54,USER219,647,290365.090,5.250169,0.146084,2631874.970,0.099142,0.110326
315,USER72,591,285030.090,5.144784,0.138854,825719.830,0.354742,0.345190
316,USER73,433,211982.684,3.701846,0.102595,233334.456,0.894628,0.908493
77,USER241,433,181739.980,3.104448,0.094324,747659.820,0.256062,0.243078
...,...,...,...,...,...,...,...,...
222,USER377,0,0.000,-0.485544,0.000000,5918.390,0.000000,0.000000
50,USER209,0,0.000,-0.485544,0.000000,971.480,0.000000,0.000000
213,USER369,0,0.000,-0.485544,0.000000,4204.430,0.000000,0.000000
212,USER368,0,0.000,-0.485544,0.000000,2811.340,0.000000,0.000000


In [55]:
# show the top 10 user who have the higher scores
locations_top_10 = locations_rating_df.nlargest(10,'locations_rating')
locations_top_10

,user,val_ds584_flag_#,err_cost_locations,std_err_cost_locations,locations_rating,org_cost_locations,val_ds584_flag_%,err_cost_locations_%
108,USER270,190,487493.270,9.144131,0.152911,487493.270,1.000000,1.000000
54,USER219,647,290365.090,5.250169,0.146084,2631874.970,0.099142,0.110326
315,USER72,591,285030.090,5.144784,0.138854,825719.830,0.354742,0.345190
316,USER73,433,211982.684,3.701846,0.102595,233334.456,0.894628,0.908493
77,USER241,433,181739.980,3.104448,0.094324,747659.820,0.256062,0.243078
138,USER3,408,187565.980,3.219532,0.093341,249960.320,0.739130,0.750383
41,USER174,363,194060.600,3.347823,0.090481,194060.600,1.000000,1.000000
63,USER227,479,108469.660,1.657107,0.079023,188810.820,0.642953,0.574489
13,USER119,335,162601.791,2.726403,0.078991,162601.791,1.000000,1.000000
88,USER252,347,154286.692,2.562151,0.077953,154286.692,1.000000,1.000000


**DASHBOARD

In [56]:
# outer join two tables by using common field "user"
dashboard_locations_df = processed_locations_df[["location", "user"]]
dashboard_locations_df = pd.merge(dashboard_locations_df, locations_rating_df
                                  , left_on="user", right_on="user" ,how="left")
# display the result
dashboard_locations_df

,location,user,val_ds584_flag_#,err_cost_locations,std_err_cost_locations,locations_rating,org_cost_locations,val_ds584_flag_%,err_cost_locations_%
0,LOCATION 8,USER38,203,134595.877,2.173189,0.057730,748376.369,0.121922,0.179851
1,LOCATION 113,USER65,88,37202.450,0.249333,0.019243,204579.380,0.176000,0.181848
2,LOCATION 66,USER219,647,290365.090,5.250169,0.146084,2631874.970,0.099142,0.110326
3,LOCATION 57,USER35,9,3604.246,-0.414347,0.001913,690234.309,0.005587,0.005222
4,LOCATION 15,USER220,15,27606.140,0.059773,0.009096,413576.203,0.027985,0.066750
...,...,...,...,...,...,...,...,...,...
115336,LOCATION 101,USER248,103,48415.300,0.470826,0.023855,301970.511,0.155120,0.160331
115337,LOCATION 67,USER58,8,4235.200,-0.401884,0.001983,1218478.340,0.003386,0.003476
115338,LOCATION 174,USER153,67,35554.480,0.216780,0.016628,94047.360,0.389535,0.378049
115339,LOCATION 218,USER224,9,2773.210,-0.430763,0.001686,644897.550,0.004302,0.004300


In [57]:
dashboard_locations_df.isnull().sum()

location                     0
user                      5545
val_ds584_flag_#          5545
err_cost_locations        5545
std_err_cost_locations    5545
locations_rating          5545
org_cost_locations        5545
val_ds584_flag_%          5545
err_cost_locations_%      5545
dtype: int64

In [58]:
# remove the missing values
dashboard_locations_df.dropna(axis=0, how='any', inplace=True)

# remove the duplicate rows
dashboard_locations_df.drop_duplicates(['location','user','locations_rating'],inplace=True)
dashboard_locations_df

,location,user,val_ds584_flag_#,err_cost_locations,std_err_cost_locations,locations_rating,org_cost_locations,val_ds584_flag_%,err_cost_locations_%
0,LOCATION 8,USER38,203,134595.877,2.173189,0.057730,748376.369,0.121922,0.179851
1,LOCATION 113,USER65,88,37202.450,0.249333,0.019243,204579.380,0.176000,0.181848
2,LOCATION 66,USER219,647,290365.090,5.250169,0.146084,2631874.970,0.099142,0.110326
3,LOCATION 57,USER35,9,3604.246,-0.414347,0.001913,690234.309,0.005587,0.005222
4,LOCATION 15,USER220,15,27606.140,0.059773,0.009096,413576.203,0.027985,0.066750
...,...,...,...,...,...,...,...,...,...
100909,LOCATION 66,USER444,2,1691.840,-0.452124,0.000669,1691.840,1.000000,1.000000
103521,LOCATION 58,USER445,0,0.000,-0.485544,0.000000,1100.000,0.000000,0.000000
104429,LOCATION 646,USER446,1,1848.700,-0.449025,0.000609,1848.700,1.000000,1.000000
108296,LOCATION 101,USER203,51,42989.240,0.363642,0.017013,43973.890,0.927273,0.977608


In [59]:
# generate new dashboard
dashboard_locations_df_1 = dashboard_locations_df.copy()

# select displayed columns
dashboard_locations_df_1 = dashboard_locations_df_1[['location', 'user', 'locations_rating'
                                                     , 'val_ds584_flag_%', 'err_cost_locations_%']]
# adjust the column names
dashboard_locations_df_1['role'] = 'Locations'
dashboard_locations_df_1.columns.values[2] = "rating"

# display the results
dashboard_locations_df_1

,location,user,rating,val_ds584_flag_%,err_cost_locations_%,role
0,LOCATION 8,USER38,0.057730,0.121922,0.179851,Locations
1,LOCATION 113,USER65,0.019243,0.176000,0.181848,Locations
2,LOCATION 66,USER219,0.146084,0.099142,0.110326,Locations
3,LOCATION 57,USER35,0.001913,0.005587,0.005222,Locations
4,LOCATION 15,USER220,0.009096,0.027985,0.066750,Locations
...,...,...,...,...,...,...
100909,LOCATION 66,USER444,0.000669,1.000000,1.000000,Locations
103521,LOCATION 58,USER445,0.000000,0.000000,0.000000,Locations
104429,LOCATION 646,USER446,0.000609,1.000000,1.000000,Locations
108296,LOCATION 101,USER203,0.017013,0.927273,0.977608,Locations


In [60]:
# show the results by score from large to small
dashboard_locations_df_1.sort_values(by='rating', ascending=False)

,location,user,rating,val_ds584_flag_%,err_cost_locations_%,role
355,LOCATION 105,USER270,0.152911,1.000000,1.000000,Locations
2,LOCATION 66,USER219,0.146084,0.099142,0.110326,Locations
164,LOCATION 71,USER72,0.138854,0.354742,0.345190,Locations
359,LOCATION 23,USER73,0.102595,0.894628,0.908493,Locations
1253,LOCATION 559,USER73,0.102595,0.894628,0.908493,Locations
...,...,...,...,...,...,...
201,LOCATION 105,USER263,0.000000,0.000000,0.000000,Locations
51209,LOCATION 1,USER93,0.000000,0.000000,0.000000,Locations
2522,LOCATION 58,USER335,0.000000,0.000000,0.000000,Locations
2343,LOCATION 218,USER110,0.000000,0.000000,0.000000,Locations


In [61]:
# select the records that rating >= 0
dashboard_locations_df_2 = dashboard_locations_df_1[dashboard_locations_df_1['rating'] >= 0]
dashboard_locations_df_2

,location,user,rating,val_ds584_flag_%,err_cost_locations_%,role
0,LOCATION 8,USER38,0.057730,0.121922,0.179851,Locations
1,LOCATION 113,USER65,0.019243,0.176000,0.181848,Locations
2,LOCATION 66,USER219,0.146084,0.099142,0.110326,Locations
3,LOCATION 57,USER35,0.001913,0.005587,0.005222,Locations
4,LOCATION 15,USER220,0.009096,0.027985,0.066750,Locations
...,...,...,...,...,...,...
100909,LOCATION 66,USER444,0.000669,1.000000,1.000000,Locations
103521,LOCATION 58,USER445,0.000000,0.000000,0.000000,Locations
104429,LOCATION 646,USER446,0.000609,1.000000,1.000000,Locations
108296,LOCATION 101,USER203,0.017013,0.927273,0.977608,Locations


In [62]:
# show the results by score from large to small
dashboard_locations_df_2.sort_values(by='rating', ascending=False)

,location,user,rating,val_ds584_flag_%,err_cost_locations_%,role
355,LOCATION 105,USER270,0.152911,1.000000,1.000000,Locations
2,LOCATION 66,USER219,0.146084,0.099142,0.110326,Locations
164,LOCATION 71,USER72,0.138854,0.354742,0.345190,Locations
359,LOCATION 23,USER73,0.102595,0.894628,0.908493,Locations
1253,LOCATION 559,USER73,0.102595,0.894628,0.908493,Locations
...,...,...,...,...,...,...
201,LOCATION 105,USER263,0.000000,0.000000,0.000000,Locations
51209,LOCATION 1,USER93,0.000000,0.000000,0.000000,Locations
2522,LOCATION 58,USER335,0.000000,0.000000,0.000000,Locations
2343,LOCATION 218,USER110,0.000000,0.000000,0.000000,Locations


In [63]:
dashboard_locations_df_2.groupby(by = ['location'])['rating'].count()

location
LOCATION 1       5
LOCATION 100     5
LOCATION 101    16
LOCATION 104     3
LOCATION 105     9
                ..
LOCATION 79      3
LOCATION 8       3
LOCATION 82      4
LOCATION 95      6
LOCATION 98      1
Name: rating, Length: 112, dtype: int64

**DATASET Receiving

In [64]:
# read excel file
role = get_execution_role()
bucket = 'dean690-dataset'
data_key3 = 'gmu_training_receiving_2.csv'
data_path3 = 's3://{}/{}'.format(bucket, data_key3)

# read csv file
data_receiving = pd.read_csv(data_path3, low_memory = False)
data_receiving.head()

data_receiving.info()

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20210312T142701 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53242 entries, 0 to 53241
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                53242 non-null  int64  
 1   BUSINESS_UNIT             53242 non-null  object 
 2   CREATION_METHOD           53242 non-null  object 
 3   OPRID                     53242 non-null  object 
 4   DOS_AMSUBTYPE_DESC        53242 non-null  object 
 5   ASSET_CLASS               53242 non-null  int64  
 6   DOS_ASSET_CL_DESCR        53242 non-null  object 
 7   ACQUISITION_DT            53242 non-null  object 
 8   DOS_AGENCY_CD             53242 non-null  object 
 9   COST                      53242 non-null  float64
 10  DOS_LOCATION_TYPE2_DESCR  53242 non-null  object 
 11  FAP_LIKE_FLAG             53242 non-null  object 
 12  MISCLASSIFIED_FAP         3485 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 5.3+ MB


In [65]:
# check missing value
data_receiving.isnull().sum()

Unnamed: 0                      0
BUSINESS_UNIT                   0
CREATION_METHOD                 0
OPRID                           0
DOS_AMSUBTYPE_DESC              0
ASSET_CLASS                     0
DOS_ASSET_CL_DESCR              0
ACQUISITION_DT                  0
DOS_AGENCY_CD                   0
COST                            0
DOS_LOCATION_TYPE2_DESCR        0
FAP_LIKE_FLAG                   0
MISCLASSIFIED_FAP           49757
dtype: int64

In [66]:
# remove duplicate rows
data_receiving.drop_duplicates(inplace=True)

***Misclassified_FAP

In [67]:
data_receiving["MISCLASSIFIED_FAP"].value_counts()

Should be FAP based on Asset Class        2379
Should not be FAP based on Asset Class     817
FAP not in Approved Location               289
Name: MISCLASSIFIED_FAP, dtype: int64

In [68]:
# fill the missing value with 0
data_receiving["MISCLASSIFIED_FAP"].fillna(0, inplace=True)

# adjust the values
data_receiving["MISCLASSIFIED_FAP"].replace(["Should be FAP based on Asset Class"
                                             , "Should not be FAP based on Asset Class"
                                             , "FAP not in Approved Location"], [1, 1, 1], inplace=True)
# display the adjusted values
data_receiving["MISCLASSIFIED_FAP"].value_counts()

0    49757
1     3485
Name: MISCLASSIFIED_FAP, dtype: int64

**CREATION_METHOD

In [69]:
data_receiving["CREATION_METHOD"].value_counts()

Purch. Req         46676
Manual Creation     6076
AM Page              357
PI Add               133
Name: CREATION_METHOD, dtype: int64

In [70]:
# adjust the values
data_receiving["CREATION_METHOD"].replace(["AM Page", "Manual Creation", "PI Add", "Purch. Req"], [1, 1, 1, 0], inplace=True)

# display the adjusted values
data_receiving["CREATION_METHOD"].value_counts()

0    46676
1     6566
Name: CREATION_METHOD, dtype: int64

**COST

In [71]:
# convert datatype to string
data_receiving["COST"] = data_receiving["COST"].astype(str)

# remove symbols in cost column
data_receiving.COST = data_receiving.COST.str.replace('[#,@,&,-]', '')
data_receiving.COST = data_receiving.COST.str.replace("'", '')

# convert datatype to float
data_receiving["COST"] = data_receiving["COST"].astype(float)

# display the result
data_receiving.head()

,Unnamed: 0,BUSINESS_UNIT,CREATION_METHOD,OPRID,DOS_AMSUBTYPE_DESC,ASSET_CLASS,DOS_ASSET_CL_DESCR,ACQUISITION_DT,DOS_AGENCY_CD,COST,DOS_LOCATION_TYPE2_DESCR,FAP_LIKE_FLAG,MISCLASSIFIED_FAP
0,1,LOCATION 62,0,USER434,Household Furniture,13303,CABINET CHINA,27.09.2019,AGENCY1,1049.63,Stock/Warehouse,Y,0
1,2,LOCATION 120,0,USER448,Maint Machinery/Equipment,46104,LIFTER,24.08.2018,AGENCY4,12874.82,Office,N,0
2,3,LOCATION 162,0,USER279,Household Furniture,15223,CHAIR HSLD DNING STD. W/ARMS,14.08.2019,AGENCY3,466.09,Stock/Warehouse,Y,0
3,4,LOCATION 165,0,USER391,Appliances,39300,TRANSFORMER,12.07.2018,AGENCY1,62.82,Stock/Warehouse,Y,0
4,5,LOCATION 59,0,USER274,Household Furniture,11305,HEADBOARD TWIN,24.01.2019,AGENCY1,405.43,Stock/Warehouse,Y,0


**Generate processed table and filter the error cost

In [72]:
# select colomns
processed_receiving_df = data_receiving[['BUSINESS_UNIT', 'OPRID', 'MISCLASSIFIED_FAP','CREATION_METHOD', 'COST']]

# rename the columns
processed_receiving_df.columns = ['location', 'user','misclf_pfm', 'cre_mthod_pfm','ori_cost']

# display the result
processed_receiving_df

,location,user,misclf_pfm,cre_mthod_pfm,ori_cost
0,LOCATION 62,USER434,0,0,1049.630
1,LOCATION 120,USER448,0,0,12874.820
2,LOCATION 162,USER279,0,0,466.090
3,LOCATION 165,USER391,0,0,62.820
4,LOCATION 59,USER274,0,0,405.430
...,...,...,...,...,...
53237,LOCATION 113,USER311,0,0,267.820
53238,LOCATION 27,USER337,0,0,1148.530
53239,LOCATION 100,USER449,0,0,731.564
53240,LOCATION 1,USER351,0,0,23.520


In [73]:
# copy the processed table
processed_receiving_df_1 = processed_receiving_df.copy()

# filter the error cost
processed_receiving_df_1.loc[(processed_receiving_df_1['misclf_pfm']>0) | (
    processed_receiving_df_1['cre_mthod_pfm']>0) , 'err_cost'] = processed_receiving_df_1['ori_cost']

# check if there is missing values
processed_receiving_df_1.isnull().sum()

location             0
user                 0
misclf_pfm           0
cre_mthod_pfm        0
ori_cost             0
err_cost         43297
dtype: int64

In [74]:
# fill nan value with 0
processed_receiving_df_1.fillna(0, inplace=True)

# check if the missing values are removed
processed_receiving_df_1.isnull().sum()

location         0
user             0
misclf_pfm       0
cre_mthod_pfm    0
ori_cost         0
err_cost         0
dtype: int64

In [75]:
# display the results
processed_receiving_df_1.head()

,location,user,misclf_pfm,cre_mthod_pfm,ori_cost,err_cost
0,LOCATION 62,USER434,0,0,1049.63,0.0
1,LOCATION 120,USER448,0,0,12874.82,0.0
2,LOCATION 162,USER279,0,0,466.09,0.0
3,LOCATION 165,USER391,0,0,62.82,0.0
4,LOCATION 59,USER274,0,0,405.43,0.0


**Calculation: Sum the transactions number of each action and the cost

In [76]:
# create new table to continue the modeling
receiving_rating_df = pd.DataFrame(columns=["user", "misclf_fap_#", "cre_mthod_#", "org_cost_receiving"
                                            , "err_cost_receiving"
                                            , "misclf_fap_%", "cre_mthod_%", "err_cost_receiving_%"])

# use sum() function to calculate total amounts in each column
# misclf_fap_# : the total transaction number of action "MISCLASSIFIED_FAP"
# cre_mthod_# : the total transaction number of action "CREATION_METHOD"
# org_cost_locations : total orginal cost by user
# err_cost_receiving : total error cost by user
# misclf_fap_% : the proportion of error transaction number of action "MISCLASSIFIED_FAP"
# cre_mthod_% : the proportion of error transaction number of action "CREATION_METHOD"
# err_cost_locations_% : the proportion of error cost by user

for index, row in processed_receiving_df_1.groupby('user'):
    sub_df = pd.DataFrame({"user": index,  "misclf_fap_#": row["misclf_pfm"].sum()
                           , "cre_mthod_#": row["cre_mthod_pfm"].sum()
                           , "org_cost_receiving": row["ori_cost"].sum()
                           , "err_cost_receiving": row["err_cost"].sum()
                           , "misclf_fap_%": row["misclf_pfm"].sum() / len(row)
                           , "cre_mthod_%": row["cre_mthod_pfm"].sum() / len(row)
                           , "err_cost_receiving_%": row["err_cost"].sum() / row["ori_cost"].sum()}, index=[0])
    receiving_rating_df = pd.concat([receiving_rating_df, sub_df], axis=0)

# display the result
receiving_rating_df

,user,misclf_fap_#,cre_mthod_#,org_cost_receiving,err_cost_receiving,misclf_fap_%,cre_mthod_%,err_cost_receiving_%
0,USER10,0,53,3577.500,3577.50,0.000000,1.000000,1.000000
0,USER101,139,11,93195.000,93195.00,1.000000,0.079137,1.000000
0,USER102,0,0,70602.914,0.00,0.000000,0.000000,0.000000
0,USER103,0,69,25281.760,25281.76,0.000000,1.000000,1.000000
0,USER109,0,18,4842.000,4842.00,0.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...
0,USER73,27,0,443700.140,14812.89,0.040419,0.000000,0.033385
0,USER80,15,78,34027.250,34027.25,0.192308,1.000000,1.000000
0,USER85,0,0,28883.000,0.00,0.000000,0.000000,0.000000
0,USER90,34,0,43868.140,18726.52,0.539683,0.000000,0.426882


In [77]:
# fill the missing value with 0
receiving_rating_df.fillna(0, inplace=True)

# display the result
receiving_rating_df

,user,misclf_fap_#,cre_mthod_#,org_cost_receiving,err_cost_receiving,misclf_fap_%,cre_mthod_%,err_cost_receiving_%
0,USER10,0,53,3577.500,3577.50,0.000000,1.000000,1.000000
0,USER101,139,11,93195.000,93195.00,1.000000,0.079137,1.000000
0,USER102,0,0,70602.914,0.00,0.000000,0.000000,0.000000
0,USER103,0,69,25281.760,25281.76,0.000000,1.000000,1.000000
0,USER109,0,18,4842.000,4842.00,0.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...
0,USER73,27,0,443700.140,14812.89,0.040419,0.000000,0.033385
0,USER80,15,78,34027.250,34027.25,0.192308,1.000000,1.000000
0,USER85,0,0,28883.000,0.00,0.000000,0.000000,0.000000
0,USER90,34,0,43868.140,18726.52,0.539683,0.000000,0.426882


In [78]:
# create a new variable to store reference columns
cost_rec_temp = receiving_rating_df[['user','org_cost_receiving'
                                     , 'misclf_fap_%', 'cre_mthod_%', 'err_cost_receiving_%']]

# filter the columns before modeling
receiving_rating_df = receiving_rating_df [['user', 'misclf_fap_#', 'cre_mthod_#', 'err_cost_receiving']]

# display the table
receiving_rating_df

,user,misclf_fap_#,cre_mthod_#,err_cost_receiving
0,USER10,0,53,3577.50
0,USER101,139,11,93195.00
0,USER102,0,0,0.00
0,USER103,0,69,25281.76
0,USER109,0,18,4842.00
...,...,...,...,...
0,USER73,27,0,14812.89
0,USER80,15,78,34027.25
0,USER85,0,0,0.00
0,USER90,34,0,18726.52


**DATA MODELING

In [79]:
# weights for role receiving
weight_misclf_fap = 0.5/15
weight_cre_method = 1/15
weight_rec_cost = 3/15

In [80]:
# Standardize the error cost
receiving_rating_df["std_err_cost_receiving"] = StandardScaler().fit_transform(
    receiving_rating_df["err_cost_receiving"].astype(float).values.reshape(-1, 1))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [81]:
# Normalizing each feature
X_2 = MinMaxScaler(feature_range=(0, 1)).fit_transform(receiving_rating_df[["misclf_fap_#", "cre_mthod_#"
                                                                            , "std_err_cost_receiving"]]) 
# assign weights for each attribute
X_2[:, 0] = weight_misclf_fap * X_2[:, 0]      # weight for misclf_fap_# 
X_2[:, 1] = weight_cre_method * X_2[:, 1]      # weight for cre_mthod_# 
X_2[:, 2] = weight_rec_cost * X_2[:, 2]      # weight for std_cost_locations


rating_2 = X_2[:, 0] + X_2[:, 1] + X_2[:,2]
receiving_rating_df["receiving_rating"] = rating_2

# display the result
receiving_rating_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,user,misclf_fap_#,cre_mthod_#,err_cost_receiving,std_err_cost_receiving,receiving_rating
0,USER10,0,53,3577.50,-0.103250,0.001257
0,USER101,139,11,93195.00,0.051578,0.021318
0,USER102,0,0,0.00,-0.109431,0.000000
0,USER103,0,69,25281.76,-0.065753,0.002184
0,USER109,0,18,4842.00,-0.101066,0.000523
...,...,...,...,...,...,...
0,USER73,27,0,14812.89,-0.083840,0.004007
0,USER80,15,78,34027.25,-0.050644,0.004621
0,USER85,0,0,0.00,-0.109431,0.000000
0,USER90,34,0,18726.52,-0.077078,0.005048


**RESULTS

In [83]:
# merge the orginal total cost to the result for reference
receiving_rating_df = pd.merge(receiving_rating_df, cost_rec_temp, on="user", how="outer")

# show the results by score from large to small
receiving_rating_df.sort_values(by='receiving_rating', ascending=False)

,user,misclf_fap_#,cre_mthod_#,err_cost_receiving,std_err_cost_receiving,receiving_rating,org_cost_receiving,misclf_fap_%,cre_mthod_%,err_cost_receiving_%
111,USER453,0,3039,7547970.71,12.930853,0.266667,7547970.710,0.000000,1.000000,1.000000
38,USER279,249,37,206313.35,0.247007,0.039612,409818.880,0.335580,0.049865,0.503426
36,USER276,58,1013,139596.99,0.131744,0.033686,1060178.194,0.020737,0.362174,0.131673
98,USER416,231,0,88428.38,0.043343,0.033267,222220.390,0.619303,0.000000,0.397931
51,USER311,186,0,40111.47,-0.040132,0.025962,224414.990,0.267626,0.000000,0.178738
...,...,...,...,...,...,...,...,...,...,...
74,USER360,0,0,0.00,-0.109431,0.000000,83524.370,0.000000,0.000000,0.000000
71,USER352,0,0,0.00,-0.109431,0.000000,566705.250,0.000000,0.000000,0.000000
69,USER349,0,0,0.00,-0.109431,0.000000,13176.350,0.000000,0.000000,0.000000
67,USER345,0,0,0.00,-0.109431,0.000000,18792.200,0.000000,0.000000,0.000000


In [96]:
# show the top 10 user who have the higher scores
receiving_top_10 = receiving_rating_df.nlargest(10,'receiving_rating')
receiving_top_10

,user,misclf_fap_#,cre_mthod_#,err_cost_receiving,std_err_cost_receiving,receiving_rating,org_cost_receiving,misclf_fap_%,cre_mthod_%,err_cost_receiving_%
111,USER453,0,3039,7547970.710,12.930853,0.266667,7547970.710,0.000000,1.000000,1.000000
38,USER279,249,37,206313.350,0.247007,0.039612,409818.880,0.335580,0.049865,0.503426
36,USER276,58,1013,139596.990,0.131744,0.033686,1060178.194,0.020737,0.362174,0.131673
98,USER416,231,0,88428.380,0.043343,0.033267,222220.390,0.619303,0.000000,0.397931
51,USER311,186,0,40111.470,-0.040132,0.025962,224414.990,0.267626,0.000000,0.178738
130,USER471,156,0,178174.250,0.198392,0.025605,238958.750,0.636735,0.000000,0.745628
107,USER449,152,83,44376.792,-0.032763,0.023345,2315108.657,0.035968,0.019640,0.019168
78,USER365,145,0,90273.430,0.046530,0.021803,121000.450,0.771277,0.000000,0.746059
70,USER351,136,108,28945.420,-0.059423,0.021342,537252.970,0.097631,0.077531,0.053877
1,USER101,139,11,93195.000,0.051578,0.021318,93195.000,1.000000,0.079137,1.000000


**DASHBOARD

In [97]:
# outer join two tables by using common field "user"
dashboard_receiving_df = processed_receiving_df[["location", "user"]]
dashboard_receiving_df = pd.merge(dashboard_receiving_df, receiving_rating_df
                                  , left_on="user", right_on="user" ,how="left")
# display the result
dashboard_receiving_df

,location,user,misclf_fap_#,cre_mthod_#,err_cost_receiving,std_err_cost_receiving,receiving_rating,org_cost_receiving,misclf_fap_%,cre_mthod_%,err_cost_receiving_%
0,LOCATION 62,USER434,0,0,0.000,-0.109431,0.000000,457073.260,0.000000,0.000000,0.000000
1,LOCATION 120,USER448,5,0,780.000,-0.108084,0.000690,1508356.740,0.015198,0.000000,0.000517
2,LOCATION 162,USER279,249,37,206313.350,0.247007,0.039612,409818.880,0.335580,0.049865,0.503426
3,LOCATION 165,USER391,0,0,0.000,-0.109431,0.000000,67701.966,0.000000,0.000000,0.000000
4,LOCATION 59,USER274,0,0,0.000,-0.109431,0.000000,245831.930,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
53237,LOCATION 113,USER311,186,0,40111.470,-0.040132,0.025962,224414.990,0.267626,0.000000,0.178738
53238,LOCATION 27,USER337,19,0,4924.800,-0.100923,0.002674,645668.890,0.022119,0.000000,0.007627
53239,LOCATION 100,USER449,152,83,44376.792,-0.032763,0.023345,2315108.657,0.035968,0.019640,0.019168
53240,LOCATION 1,USER351,136,108,28945.420,-0.059423,0.021342,537252.970,0.097631,0.077531,0.053877


In [98]:
dashboard_receiving_df.isnull().sum()

location                  0
user                      0
misclf_fap_#              0
cre_mthod_#               0
err_cost_receiving        0
std_err_cost_receiving    0
receiving_rating          0
org_cost_receiving        0
misclf_fap_%              0
cre_mthod_%               0
err_cost_receiving_%      0
dtype: int64

In [99]:
dashboard_receiving_df.drop_duplicates(['location','user','receiving_rating'],inplace=True)
dashboard_receiving_df

,location,user,misclf_fap_#,cre_mthod_#,err_cost_receiving,std_err_cost_receiving,receiving_rating,org_cost_receiving,misclf_fap_%,cre_mthod_%,err_cost_receiving_%
0,LOCATION 62,USER434,0,0,0.00,-0.109431,0.000000,457073.260,0.000000,0.000000,0.000000
1,LOCATION 120,USER448,5,0,780.00,-0.108084,0.000690,1508356.740,0.015198,0.000000,0.000517
2,LOCATION 162,USER279,249,37,206313.35,0.247007,0.039612,409818.880,0.335580,0.049865,0.503426
3,LOCATION 165,USER391,0,0,0.00,-0.109431,0.000000,67701.966,0.000000,0.000000,0.000000
4,LOCATION 59,USER274,0,0,0.00,-0.109431,0.000000,245831.930,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
7588,LOCATION 109,USER66,0,0,0.00,-0.109431,0.000000,48785.160,0.000000,0.000000,0.000000
8081,LOCATION 165,USER496,0,0,0.00,-0.109431,0.000000,3209.100,0.000000,0.000000,0.000000
8852,LOCATION 130,USER151,9,0,9899.91,-0.092327,0.001467,9899.910,1.000000,0.000000,1.000000
9323,LOCATION 162,USER382,0,0,0.00,-0.109431,0.000000,3134.890,0.000000,0.000000,0.000000


In [100]:
# generate new dashboard
dashboard_receiving_df_1 = dashboard_receiving_df.copy()

# select displayed columns
dashboard_receiving_df_1 = dashboard_receiving_df_1[['location', 'user','receiving_rating'
                                                     , 'misclf_fap_%', 'cre_mthod_%', 'err_cost_receiving_%']]
# adjust the column names
dashboard_receiving_df_1['role'] = 'Receiving'
dashboard_receiving_df_1.columns.values[2] = "rating"

# display the results
dashboard_receiving_df_1

,location,user,rating,misclf_fap_%,cre_mthod_%,err_cost_receiving_%,role
0,LOCATION 62,USER434,0.000000,0.000000,0.000000,0.000000,Receiving
1,LOCATION 120,USER448,0.000690,0.015198,0.000000,0.000517,Receiving
2,LOCATION 162,USER279,0.039612,0.335580,0.049865,0.503426,Receiving
3,LOCATION 165,USER391,0.000000,0.000000,0.000000,0.000000,Receiving
4,LOCATION 59,USER274,0.000000,0.000000,0.000000,0.000000,Receiving
...,...,...,...,...,...,...,...
7588,LOCATION 109,USER66,0.000000,0.000000,0.000000,0.000000,Receiving
8081,LOCATION 165,USER496,0.000000,0.000000,0.000000,0.000000,Receiving
8852,LOCATION 130,USER151,0.001467,1.000000,0.000000,1.000000,Receiving
9323,LOCATION 162,USER382,0.000000,0.000000,0.000000,0.000000,Receiving


In [101]:
# show the results by score from large to small
dashboard_receiving_df_1.sort_values(by='rating', ascending=False)

,location,user,rating,misclf_fap_%,cre_mthod_%,err_cost_receiving_%,role
30,LOCATION 107,USER453,0.266667,0.000000,1.000000,1.000000,Receiving
2,LOCATION 162,USER279,0.039612,0.335580,0.049865,0.503426,Receiving
34,LOCATION 101,USER276,0.033686,0.020737,0.362174,0.131673,Receiving
7167,LOCATION 628,USER276,0.033686,0.020737,0.362174,0.131673,Receiving
83,LOCATION 209,USER416,0.033267,0.619303,0.000000,0.397931,Receiving
...,...,...,...,...,...,...,...
239,LOCATION 14,USER429,0.000000,0.000000,0.000000,0.000000,Receiving
232,LOCATION 95,USER473,0.000000,0.000000,0.000000,0.000000,Receiving
223,LOCATION 226,USER437,0.000000,0.000000,0.000000,0.000000,Receiving
221,LOCATION 174,USER57,0.000000,0.000000,0.000000,0.000000,Receiving


In [102]:
# select the records that rating >= 0
dashboard_receiving_df_1 = dashboard_receiving_df_1[dashboard_receiving_df_1['rating'] >= 0]
dashboard_receiving_df_1

,location,user,rating,misclf_fap_%,cre_mthod_%,err_cost_receiving_%,role
0,LOCATION 62,USER434,0.000000,0.000000,0.000000,0.000000,Receiving
1,LOCATION 120,USER448,0.000690,0.015198,0.000000,0.000517,Receiving
2,LOCATION 162,USER279,0.039612,0.335580,0.049865,0.503426,Receiving
3,LOCATION 165,USER391,0.000000,0.000000,0.000000,0.000000,Receiving
4,LOCATION 59,USER274,0.000000,0.000000,0.000000,0.000000,Receiving
...,...,...,...,...,...,...,...
7588,LOCATION 109,USER66,0.000000,0.000000,0.000000,0.000000,Receiving
8081,LOCATION 165,USER496,0.000000,0.000000,0.000000,0.000000,Receiving
8852,LOCATION 130,USER151,0.001467,1.000000,0.000000,1.000000,Receiving
9323,LOCATION 162,USER382,0.000000,0.000000,0.000000,0.000000,Receiving


In [103]:
dashboard_receiving_df_1.groupby(by = ['location'])['rating'].count()

location
LOCATION 1      1
LOCATION 100    5
LOCATION 101    4
LOCATION 104    5
LOCATION 105    5
               ..
LOCATION 71     2
LOCATION 77     1
LOCATION 8      2
LOCATION 82     2
LOCATION 95     3
Name: rating, Length: 87, dtype: int64

**SUMMARY_Adjusted

In [104]:
disposals_rating_df.shape

(218, 12)

In [105]:
locations_rating_df.shape

(330, 8)

In [106]:
receiving_rating_df.shape

(169, 10)

In [107]:
# outer join three table, by using common field "user"
Rating = pd.merge(disposals_rating_df, locations_rating_df, on="user", how="outer")
Rating = pd.merge(Rating, receiving_rating_df, on="user", how="outer")

# display the results
Rating

,user,scan_type_#,ret_date_#,disp_doc_#,err_cost_disposals,std_err_cost_disposals,disposals_rating,org_cost_disposals,scan_type_%,ret_date_%,...,err_cost_locations_%,misclf_fap_#,cre_mthod_#,err_cost_receiving,std_err_cost_receiving,receiving_rating,org_cost_receiving,misclf_fap_%,cre_mthod_%,err_cost_receiving_%
0,USER1,26.0,377,87,162311.120,0.449160,0.060155,353152.910,0.030481,0.441970,...,0.076912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USER10,99.0,208,0,142545.810,0.323838,0.051490,376585.870,0.131300,0.275862,...,0.271798,0.0,53.0,3577.5,-0.103250,0.001257,3577.500,0.0,1.000000,1.0
2,USER100,14.5,83,0,202081.708,0.701325,0.061813,279244.708,0.064159,0.367257,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USER101,42.0,140,0,76011.120,-0.098024,0.027486,142507.140,0.172840,0.576132,...,0.065588,139.0,11.0,93195.0,0.051578,0.021318,93195.000,1.0,0.079137,1.0
4,USER102,24.0,93,0,46378.746,-0.285908,0.016842,170262.852,0.049383,0.191358,...,0.126391,0.0,0.0,0.0,-0.109431,0.000000,70602.914,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,USER492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,-0.109431,0.000000,854.720,0.0,0.000000,0.0
491,USER493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,-0.109431,0.000000,6504.680,0.0,0.000000,0.0
492,USER494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,-0.109431,0.000000,16266.400,0.0,0.000000,0.0
493,USER495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,-0.109431,0.000000,30249.000,0.0,0.000000,0.0


In [109]:
# check if there is missing values
# missing value here has meaning, which means the user doesnt have specific role
Rating.isnull().sum()

user                        0
scan_type_#               277
ret_date_#                277
disp_doc_#                277
err_cost_disposals        277
std_err_cost_disposals    277
disposals_rating          277
org_cost_disposals        277
scan_type_%               277
ret_date_%                277
disp_doc_%                277
err_cost_disposals_%      277
val_ds584_flag_#          165
err_cost_locations        165
std_err_cost_locations    165
locations_rating          165
org_cost_locations        165
val_ds584_flag_%          165
err_cost_locations_%      165
misclf_fap_#              326
cre_mthod_#               326
err_cost_receiving        326
std_err_cost_receiving    326
receiving_rating          326
org_cost_receiving        326
misclf_fap_%              326
cre_mthod_%               326
err_cost_receiving_%      326
dtype: int64

In [110]:
# fill nan value with 0
Rating.fillna(0, inplace=True)

# check if the missing values are removed
Rating.isnull().sum()

user                      0
scan_type_#               0
ret_date_#                0
disp_doc_#                0
err_cost_disposals        0
std_err_cost_disposals    0
disposals_rating          0
org_cost_disposals        0
scan_type_%               0
ret_date_%                0
disp_doc_%                0
err_cost_disposals_%      0
val_ds584_flag_#          0
err_cost_locations        0
std_err_cost_locations    0
locations_rating          0
org_cost_locations        0
val_ds584_flag_%          0
err_cost_locations_%      0
misclf_fap_#              0
cre_mthod_#               0
err_cost_receiving        0
std_err_cost_receiving    0
receiving_rating          0
org_cost_receiving        0
misclf_fap_%              0
cre_mthod_%               0
err_cost_receiving_%      0
dtype: int64

In [111]:
# remove duplicate rows
Rating.drop_duplicates(inplace=True)
Rating

,user,scan_type_#,ret_date_#,disp_doc_#,err_cost_disposals,std_err_cost_disposals,disposals_rating,org_cost_disposals,scan_type_%,ret_date_%,...,err_cost_locations_%,misclf_fap_#,cre_mthod_#,err_cost_receiving,std_err_cost_receiving,receiving_rating,org_cost_receiving,misclf_fap_%,cre_mthod_%,err_cost_receiving_%
0,USER1,26.0,377,87,162311.120,0.449160,0.060155,353152.910,0.030481,0.441970,...,0.076912,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.0
1,USER10,99.0,208,0,142545.810,0.323838,0.051490,376585.870,0.131300,0.275862,...,0.271798,0.0,53.0,3577.5,-0.103250,0.001257,3577.500,0.0,1.000000,1.0
2,USER100,14.5,83,0,202081.708,0.701325,0.061813,279244.708,0.064159,0.367257,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.0
3,USER101,42.0,140,0,76011.120,-0.098024,0.027486,142507.140,0.172840,0.576132,...,0.065588,139.0,11.0,93195.0,0.051578,0.021318,93195.000,1.0,0.079137,1.0
4,USER102,24.0,93,0,46378.746,-0.285908,0.016842,170262.852,0.049383,0.191358,...,0.126391,0.0,0.0,0.0,-0.109431,0.000000,70602.914,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,USER492,0.0,0,0,0.000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,-0.109431,0.000000,854.720,0.0,0.000000,0.0
491,USER493,0.0,0,0,0.000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,-0.109431,0.000000,6504.680,0.0,0.000000,0.0
492,USER494,0.0,0,0,0.000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,-0.109431,0.000000,16266.400,0.0,0.000000,0.0
493,USER495,0.0,0,0,0.000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,-0.109431,0.000000,30249.000,0.0,0.000000,0.0


**Calculate total rating

In [112]:
# calculate total rating
Rating['total_rating'] = Rating['disposals_rating'] + Rating['locations_rating'] + Rating['receiving_rating']

# show the results by score from large to small
Rating.sort_values(by='total_rating', ascending=False)

,user,scan_type_#,ret_date_#,disp_doc_#,err_cost_disposals,std_err_cost_disposals,disposals_rating,org_cost_disposals,scan_type_%,ret_date_%,...,misclf_fap_#,cre_mthod_#,err_cost_receiving,std_err_cost_receiving,receiving_rating,org_cost_receiving,misclf_fap_%,cre_mthod_%,err_cost_receiving_%,total_rating
157,USER44,105.0,1507,249,1135946.311,6.622482,0.379776,1764655.665,0.033302,0.477958,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.379776
178,USER63,1242.5,1594,6,912374.690,5.204929,0.367952,1154820.350,0.445181,0.571121,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.367952
179,USER64,167.5,1321,0,1016052.240,5.862295,0.334763,1386107.310,0.096486,0.760945,...,0.0,0.0,0.00,-0.109431,0.000000,31188.12,0.0,0.0,0.0,0.357929
167,USER53,195.5,1538,1786,628189.278,3.403055,0.293655,660450.721,0.096022,0.755403,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.314840
451,USER453,0.0,0,0,0.000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.0,3039.0,7547970.71,12.930853,0.266667,7547970.71,0.0,1.0,1.0,0.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,USER133,0.0,0,0,0.000,-0.579972,0.000000,16469.600,0.000000,0.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000
457,USER459,0.0,0,0,0.000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.0,0.0,0.00,-0.109431,0.000000,159498.09,0.0,0.0,0.0,0.000000
315,USER317,0.0,0,0,0.000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000
313,USER315,0.0,0,0,0.000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000


In [113]:
# show the top 10 user who have the higher scores
Rating_top_10 = Rating.nlargest(10,'total_rating')
Rating_top_10

,user,scan_type_#,ret_date_#,disp_doc_#,err_cost_disposals,std_err_cost_disposals,disposals_rating,org_cost_disposals,scan_type_%,ret_date_%,...,misclf_fap_#,cre_mthod_#,err_cost_receiving,std_err_cost_receiving,receiving_rating,org_cost_receiving,misclf_fap_%,cre_mthod_%,err_cost_receiving_%,total_rating
157,USER44,105.0,1507,249,1135946.311,6.622482,0.379776,1764655.665,0.033302,0.477958,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.379776
178,USER63,1242.5,1594,6,912374.690,5.204929,0.367952,1154820.350,0.445181,0.571121,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.367952
179,USER64,167.5,1321,0,1016052.240,5.862295,0.334763,1386107.310,0.096486,0.760945,...,0.0,0.0,0.00,-0.109431,0.000000,31188.12,0.0,0.000000,0.000000,0.357929
167,USER53,195.5,1538,1786,628189.278,3.403055,0.293655,660450.721,0.096022,0.755403,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.314840
451,USER453,0.0,0,0,0.000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.0,3039.0,7547970.71,12.930853,0.266667,7547970.71,0.0,1.000000,1.000000,0.266667
149,USER37,432.0,1019,107,551575.870,2.917289,0.210337,650603.510,0.313043,0.738406,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.234780
152,USER4,258.5,928,0,584541.480,3.126307,0.204804,954106.460,0.136196,0.488936,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.204804
163,USER5,123.5,845,0,469734.100,2.398372,0.162136,799476.260,0.076141,0.520962,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.196586
268,USER270,0.0,0,0,0.000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.0,278.0,101226.60,0.065453,0.008781,547830.58,0.0,0.247551,0.184777,0.161692
186,USER70,150.5,922,0,452719.056,2.290488,0.160202,1557018.824,0.043826,0.268492,...,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.160202


In [114]:
# generate new table
Rating_all = Rating[['user', 'disposals_rating', 'locations_rating', 'receiving_rating', 'total_rating']]

# display if the user has a specific role, if he has a role, the value = 1
Rating_all['disposal_role'] = [1 if d > 0 else 0  for d in Rating["org_cost_disposals"]]
Rating_all['locations_role'] = [1 if d > 0 else 0  for d in Rating["org_cost_locations"]]
Rating_all['receiving_role'] = [1 if d > 0 else 0  for d in Rating["org_cost_receiving"]]

# display the results
Rating_all.sort_values(by='total_rating', ascending=False)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,user,disposals_rating,locations_rating,receiving_rating,total_rating,disposal_role,locations_role,receiving_role
157,USER44,0.379776,0.000000,0.000000,0.379776,1,0,0
178,USER63,0.367952,0.000000,0.000000,0.367952,1,0,0
179,USER64,0.334763,0.023166,0.000000,0.357929,1,1,1
167,USER53,0.293655,0.021185,0.000000,0.314840,1,1,0
451,USER453,0.000000,0.000000,0.266667,0.266667,0,0,1
...,...,...,...,...,...,...,...,...
38,USER133,0.000000,0.000000,0.000000,0.000000,1,0,0
457,USER459,0.000000,0.000000,0.000000,0.000000,0,0,1
315,USER317,0.000000,0.000000,0.000000,0.000000,0,1,0
313,USER315,0.000000,0.000000,0.000000,0.000000,0,1,0


In [115]:
# show the top 10 user who have the higher scores
total_rating_top_10 = Rating_all.nlargest(10,'total_rating')
total_rating_top_10

,user,disposals_rating,locations_rating,receiving_rating,total_rating,disposal_role,locations_role,receiving_role
157,USER44,0.379776,0.000000,0.000000,0.379776,1,0,0
178,USER63,0.367952,0.000000,0.000000,0.367952,1,0,0
179,USER64,0.334763,0.023166,0.000000,0.357929,1,1,1
167,USER53,0.293655,0.021185,0.000000,0.314840,1,1,0
451,USER453,0.000000,0.000000,0.266667,0.266667,0,0,1
149,USER37,0.210337,0.024443,0.000000,0.234780,1,1,0
152,USER4,0.204804,0.000000,0.000000,0.204804,1,0,0
163,USER5,0.162136,0.034450,0.000000,0.196586,1,1,0
268,USER270,0.000000,0.152911,0.008781,0.161692,0,1,1
186,USER70,0.160202,0.000000,0.000000,0.160202,1,0,0


In [116]:
# create new table
location_disposals_rating_df = pd.DataFrame(columns=["location", "location_rating", "scan_type_%", 
                                                              "ret_date_%", "disp_doc_%", "err_cost_disposals_%", "role"])
# use sum() function to calculate total amounts in each column
# group by location

for index, row in dashboard_disposals_df_1.groupby('location'):
    sub_df = pd.DataFrame({"location": index,  "location_rating": row["rating"].sum()
                           , "scan_type_%": row["scan_type_%"].sum()
                           , "ret_date_%": row["ret_date_%"].sum()
                           , "disp_doc_%": row["disp_doc_%"].sum()
                           , "err_cost_disposals_%": row["err_cost_disposals_%"].sum()
                          , "role": "Disposals"}, index=[0])
    location_disposals_rating_df = pd.concat([location_disposals_rating_df, sub_df], axis=0)

# display the result
location_disposals_rating_df

,location,location_rating,scan_type_%,ret_date_%,disp_doc_%,err_cost_disposals_%,role
0,LOCATION 1,0.040371,0.171553,1.338190,0.000000,1.330836,Disposals
0,LOCATION 100,0.217768,1.174970,2.265682,0.012868,2.661610,Disposals
0,LOCATION 101,0.381283,0.300635,1.360217,0.877210,1.462682,Disposals
0,LOCATION 104,0.056383,0.103093,0.474227,0.000000,0.593446,Disposals
0,LOCATION 105,0.151336,0.101551,0.464732,0.000000,0.558056,Disposals
...,...,...,...,...,...,...,...
0,LOCATION 79,0.002434,0.093750,0.175000,0.000000,0.128803,Disposals
0,LOCATION 8,0.144921,0.245856,7.004678,0.003264,6.915568,Disposals
0,LOCATION 82,0.036990,0.028767,0.283562,0.000000,0.281841,Disposals
0,LOCATION 95,0.105973,0.123013,0.708630,0.004777,0.923005,Disposals


In [117]:
# show the top 10 location in role disposals
location_disposals_rating_top_10 = location_disposals_rating_df.nlargest(10,'location_rating')
location_disposals_rating_top_10

,location,location_rating,scan_type_%,ret_date_%,disp_doc_%,err_cost_disposals_%,role
0,LOCATION 141,0.579829,1.182614,5.478864,0.000000,7.267088,Disposals
0,LOCATION 120,0.436837,0.176244,1.346203,0.000000,1.354732,Disposals
0,LOCATION 27,0.429295,0.945181,0.824968,0.002150,1.699408,Disposals
0,LOCATION 131,0.397139,0.091812,2.877748,0.078972,2.824332,Disposals
0,LOCATION 637,0.388949,0.055774,0.848744,0.078972,1.008706,Disposals
0,LOCATION 101,0.381283,0.300635,1.360217,0.877210,1.462682,Disposals
0,LOCATION 100,0.217768,1.174970,2.265682,0.012868,2.661610,Disposals
0,LOCATION 30,0.210337,0.313043,0.738406,0.077536,0.847791,Disposals
0,LOCATION 124,0.209130,0.270811,1.104320,0.000000,1.431355,Disposals
0,LOCATION 66,0.181764,0.115464,0.686620,0.000000,0.705501,Disposals


In [118]:
# show the top 10 location in role disposals
location_disposals_rating_top_10 = location_disposals_rating_df.nlargest(10,'location_rating')
location_disposals_rating_top_10

,location,location_rating,scan_type_%,ret_date_%,disp_doc_%,err_cost_disposals_%,role
0,LOCATION 141,0.579829,1.182614,5.478864,0.000000,7.267088,Disposals
0,LOCATION 120,0.436837,0.176244,1.346203,0.000000,1.354732,Disposals
0,LOCATION 27,0.429295,0.945181,0.824968,0.002150,1.699408,Disposals
0,LOCATION 131,0.397139,0.091812,2.877748,0.078972,2.824332,Disposals
0,LOCATION 637,0.388949,0.055774,0.848744,0.078972,1.008706,Disposals
0,LOCATION 101,0.381283,0.300635,1.360217,0.877210,1.462682,Disposals
0,LOCATION 100,0.217768,1.174970,2.265682,0.012868,2.661610,Disposals
0,LOCATION 30,0.210337,0.313043,0.738406,0.077536,0.847791,Disposals
0,LOCATION 124,0.209130,0.270811,1.104320,0.000000,1.431355,Disposals
0,LOCATION 66,0.181764,0.115464,0.686620,0.000000,0.705501,Disposals


In [119]:
# create new table
location_location_rating_df = pd.DataFrame(columns=["location", "location_rating", "val_ds584_flag_%", 
                                                              "err_cost_locations_%", "role"])
# use sum() function to calculate total amounts in each column
# group by location

for index, row in dashboard_locations_df_1.groupby('location'):
    sub_df = pd.DataFrame({"location": index,  "location_rating": row["rating"].sum()
                           , "val_ds584_flag_%": row["val_ds584_flag_%"].sum()
                           , "err_cost_locations_%": row["err_cost_locations_%"].sum()
                          , "role": "Locations"}, index=[0])
    location_location_rating_df = pd.concat([location_location_rating_df, sub_df], axis=0)

# display the result
location_location_rating_df

,location,location_rating,val_ds584_flag_%,err_cost_locations_%,role
0,LOCATION 1,0.003488,1.023301,1.017164,Locations
0,LOCATION 100,0.015024,0.118481,0.092487,Locations
0,LOCATION 101,0.134616,8.291312,8.843908,Locations
0,LOCATION 104,0.086231,2.107659,2.114560,Locations
0,LOCATION 105,0.195007,4.333675,4.539379,Locations
...,...,...,...,...,...
0,LOCATION 79,0.085384,3.000000,3.000000,Locations
0,LOCATION 8,0.059088,1.538589,1.695408,Locations
0,LOCATION 82,0.195850,3.690350,3.684310,Locations
0,LOCATION 95,0.035488,4.180661,4.270576,Locations


In [120]:
# show the top 10 location in role location
location_location_rating_top_10 = location_location_rating_df.nlargest(10,'location_rating')
location_location_rating_top_10

,location,location_rating,val_ds584_flag_%,err_cost_locations_%,role
0,LOCATION 66,0.236908,4.135721,4.161332,Locations
0,LOCATION 82,0.195850,3.690350,3.684310,Locations
0,LOCATION 105,0.195007,4.333675,4.539379,Locations
0,LOCATION 194,0.166038,4.658064,4.626810,Locations
0,LOCATION 71,0.138854,0.354742,0.345190,Locations
0,LOCATION 101,0.134616,8.291312,8.843908,Locations
0,LOCATION 39,0.122111,1.798806,1.441300,Locations
0,LOCATION 55,0.112324,1.441474,1.221211,Locations
0,LOCATION 23,0.109900,1.894628,1.908493,Locations
0,LOCATION 166,0.107270,1.490852,1.411954,Locations


In [121]:
# create new table
location_receiving_rating_df = pd.DataFrame(columns=["location", "location_rating", "misclf_fap_%", "cre_mthod_%"
                                                     ,"err_cost_receiving_%", "role"])

# use sum() function to calculate total amounts in each column
# group by location

for index, row in dashboard_receiving_df_1.groupby('location'):
    sub_df = pd.DataFrame({"location": index,  "location_rating": row["rating"].sum()
                           , "misclf_fap_%": row["misclf_fap_%"].sum()
                           , "cre_mthod_%": row["cre_mthod_%"].sum()
                           ,"err_cost_receiving_%":row["err_cost_receiving_%"].sum()
                          , "role": "Receiving"}, index=[0])
    location_receiving_rating_df = pd.concat([location_receiving_rating_df, sub_df], axis=0)

# display the result
location_receiving_rating_df

,location,location_rating,misclf_fap_%,cre_mthod_%,err_cost_receiving_%,role
0,LOCATION 1,0.021342,0.097631,0.077531,0.053877,Receiving
0,LOCATION 100,0.031499,0.035968,1.764352,1.359903,Receiving
0,LOCATION 101,0.033686,0.020737,0.362174,0.131673,Receiving
0,LOCATION 104,0.004523,0.247670,1.000000,1.016288,Receiving
0,LOCATION 105,0.009726,0.000000,0.447551,0.353263,Receiving
...,...,...,...,...,...,...
0,LOCATION 71,0.005659,0.431034,1.000000,1.000000,Receiving
0,LOCATION 77,0.005725,1.000000,0.000000,1.000000,Receiving
0,LOCATION 8,0.001964,0.042945,0.000000,0.024308,Receiving
0,LOCATION 82,0.017491,0.690365,0.007812,0.365497,Receiving


In [122]:
# show the top 10 location in role receiving
location_receiving_rating_top_10 = location_receiving_rating_df.nlargest(10,'location_rating')
location_receiving_rating_top_10

,location,location_rating,misclf_fap_%,cre_mthod_%,err_cost_receiving_%,role
0,LOCATION 107,0.274651,0.000000,2.000000,2.000000,Receiving
0,LOCATION 209,0.042244,1.009714,0.000000,0.866786,Receiving
0,LOCATION 162,0.039612,0.335580,0.049865,0.503426,Receiving
0,LOCATION 101,0.033686,0.020737,0.362174,0.131673,Receiving
0,LOCATION 628,0.033686,0.020737,0.362174,0.131673,Receiving
0,LOCATION 100,0.031499,0.035968,1.764352,1.359903,Receiving
0,LOCATION 113,0.030872,0.267626,0.838889,0.952169,Receiving
0,LOCATION 2,0.028355,0.727163,0.405941,1.501604,Receiving
0,LOCATION 137,0.027810,0.710467,0.000000,0.746622,Receiving
0,LOCATION 11,0.025660,1.073986,0.079137,1.011622,Receiving


In [123]:
Location_Rating = pd.merge(location_disposals_rating_df, location_location_rating_df, on="location", how="outer")
Location_Rating = pd.merge(Location_Rating, location_receiving_rating_df, on="location", how="outer")

# fill nan value with 0
Location_Rating.fillna(0, inplace=True)

# check if the missing values are removed
Location_Rating.isnull().sum()

location                0
location_rating_x       0
scan_type_%             0
ret_date_%              0
disp_doc_%              0
err_cost_disposals_%    0
role_x                  0
location_rating_y       0
val_ds584_flag_%        0
err_cost_locations_%    0
role_y                  0
location_rating         0
misclf_fap_%            0
cre_mthod_%             0
err_cost_receiving_%    0
role                    0
dtype: int64

In [124]:
# calculate the total rating
Location_Rating["total_rating"] = Location_Rating["location_rating_x"] + Location_Rating["location_rating_y"
                                                                                      ] + Location_Rating["location_rating"]
# drop the sepearte rating
Location_Rating.drop(["location_rating_x", "location_rating_y", "location_rating"],axis=1, inplace=True)
Location_Rating

,location,scan_type_%,ret_date_%,disp_doc_%,err_cost_disposals_%,role_x,val_ds584_flag_%,err_cost_locations_%,role_y,misclf_fap_%,cre_mthod_%,err_cost_receiving_%,role,total_rating
0,LOCATION 1,0.171553,1.338190,0.000000,1.330836,Disposals,1.023301,1.017164,Locations,0.097631,0.077531,0.053877,Receiving,0.065201
1,LOCATION 100,1.174970,2.265682,0.012868,2.661610,Disposals,0.118481,0.092487,Locations,0.035968,1.764352,1.359903,Receiving,0.264291
2,LOCATION 101,0.300635,1.360217,0.877210,1.462682,Disposals,8.291312,8.843908,Locations,0.020737,0.362174,0.131673,Receiving,0.549584
3,LOCATION 104,0.103093,0.474227,0.000000,0.593446,Disposals,2.107659,2.114560,Locations,0.247670,1.000000,1.016288,Receiving,0.147136
4,LOCATION 105,0.101551,0.464732,0.000000,0.558056,Disposals,4.333675,4.539379,Locations,0.000000,0.447551,0.353263,Receiving,0.356069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,LOCATION 659,0.000000,0.000000,0.000000,0.000000,0,0.008952,0.048634,Locations,0.000000,0.213235,0.006990,Receiving,0.013782
129,LOCATION 660,0.000000,0.000000,0.000000,0.000000,0,0.280000,0.322878,Locations,0.000000,0.000000,0.000000,0,0.001987
130,LOCATION 661,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,Locations,0.000000,0.000000,0.000000,0,0.000000
131,LOCATION 662,0.000000,0.000000,0.000000,0.000000,0,0.115385,0.173501,Locations,0.000000,0.000000,0.000000,0,0.001396


In [125]:
# look for location id
Location_Rating["l"] = [int(s[9:]) for s in Location_Rating["location"].values]
# sort the location id from small to large
Location_Rating = Location_Rating.sort_values(by="l")
Location_Rating.drop("l", axis=1, inplace=True)
# show the results by score from large to small
Location_Rating.sort_values(by='total_rating', ascending=False)

,location,scan_type_%,ret_date_%,disp_doc_%,err_cost_disposals_%,role_x,val_ds584_flag_%,err_cost_locations_%,role_y,misclf_fap_%,cre_mthod_%,err_cost_receiving_%,role,total_rating
15,LOCATION 141,1.182614,5.478864,0.000000,7.267088,Disposals,0.896684,0.752550,Locations,0.000000,2.133333,2.183163,Receiving,0.675190
2,LOCATION 101,0.300635,1.360217,0.877210,1.462682,Disposals,8.291312,8.843908,Locations,0.020737,0.362174,0.131673,Receiving,0.549584
8,LOCATION 120,0.176244,1.346203,0.000000,1.354732,Disposals,0.224165,0.188925,Locations,0.015198,0.000000,0.000517,Receiving,0.479570
64,LOCATION 27,0.945181,0.824968,0.002150,1.699408,Disposals,1.125784,1.096320,Locations,0.171618,0.000000,0.042453,Receiving,0.456897
11,LOCATION 131,0.091812,2.877748,0.078972,2.824332,Disposals,0.112050,0.112563,Locations,0.058760,0.080460,0.120937,Receiving,0.448316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,LOCATION 204,0.000000,0.181818,0.272727,0.352802,Disposals,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.001507
125,LOCATION 656,0.000000,0.000000,0.000000,0.000000,0,0.115385,0.173501,Locations,0.000000,0.000000,0.000000,0,0.001396
127,LOCATION 658,0.000000,0.000000,0.000000,0.000000,0,0.115385,0.173501,Locations,0.000000,0.000000,0.000000,0,0.001396
131,LOCATION 662,0.000000,0.000000,0.000000,0.000000,0,0.115385,0.173501,Locations,0.000000,0.000000,0.000000,0,0.001396


In [126]:
#Export table to s3 to create dashboard

processed_disposals_df_1.to_csv('s3://dean690-dataset/table.processed_disposals_df_1.0412.csv', index=False)
processed_receiving_df_1.to_csv('s3://dean690-dataset/table.processed_receiving_df_1.0412.csv', index=False)
processed_locations_df_1.to_csv('s3://dean690-dataset/table.processed_locations_df_1.0412.csv', index=False)
Rating.to_csv('s3://dean690-dataset/table.Rating.0412.csv', index=False)
locations_rating_df.to_csv('s3://dean690-dataset/table.Location_Rating.0412.csv', index=False)